In [13]:
from zipfile import ZipFile
from io import TextIOWrapper
loc = '../../data/raw_data/GTFS_KRK_T.zip'

In [89]:
class Stop:
    def __init__(self, stop_id, name, lat, lon):
        self.stop_id = stop_id
        self.name = name
        self.lat = lat
        self.lon = lon
        
        self.beg = 0
        self.norm = 0
        self.end = 0
        
    
class RouteVersion:
    def __init__(self, route_id, beg, end):
        self.route_id = route_id
        self.beg = beg
        self.end = end
        self.count = 0
        
    def __str__(self):
        return f'{self.count} x {self.beg} - {self.end}'
        
    
class Route:
    def __init__(self, route_id, name):
        self.route_id = route_id
        self.name = name
        
        self.trip_ids = set()
        self.route_versions = dict()
        
    def add_trip(self, trip_id, beg, end):
        self.trip_ids.add(trip_id)
        if beg + end not in self.route_versions:
            self.route_versions[beg + end] = RouteVersion(self.route_id, beg, end)
        self.route_versions[beg + end].count += 1
    
    
class Trip:
    def __init__(self, trip_id, route_id, service_id, trip_headsign, direction_id, block_id):
        self.trip_id = trip_id
        self.route_id = route_id
        self.service_id = service_id
        self.trip_headsign = trip_headsign
        self.direction_id = direction_id
        self.block_id = block_id
        
        self.stop_times = []
    
    def add_stop(self, time, stop_seq, stop_id):
        self.stop_times.append((time, stop_seq, stop_id))
        
    def finalise(self):
        self.stop_times.sort()
        beg = stop_list[self.stop_times[0][2]].name
        end = stop_list[self.stop_times[-1][2]].name
        route_list[self.route_id].add_trip(self.trip_id, beg, end)
        
    def __str__(self):
        msg = f'{route_list[self.route_id].name}: {self.trip_id}\n'
        msg += f'{self.trip_headsign} (dir: {self.direction_id}), {services[self.service_id]}\n\n';
        for time, seq, stop_id in self.stop_times:
            msg += f'{time} {stop_list[stop_id].name}({seq})\n'
        msg += '\n';
        return msg;

In [90]:
with ZipFile(loc) as my_zip:
    stop_list = dict()
    stop_desc = dict()
    with TextIOWrapper(my_zip.open('stops.txt')) as file:
        s = file.readline().strip('\n').split(',')
        for i in range(len(s)):
            stop_desc[s[i]] = i

        s = file.readline().strip('\n')
        while s:
            s = s.split(',')
            stop_id = s[stop_desc['stop_id']]
            name = s[stop_desc['stop_name']].strip('\"')
            lat = float(s[stop_desc['stop_lat']])
            lon = float(s[stop_desc['stop_lon']])

            stop_list[stop_id] = Stop(stop_id, name, lat, lon)
            s = file.readline().strip('\n')

    route_list = dict()
    route_desc = dict()
    with TextIOWrapper(my_zip.open('routes.txt')) as file:
        s = file.readline().strip('\n').split(',')
        for i in range(len(s)):
            route_desc[s[i]] = i

        s = file.readline().strip('\n')
        while s:
            s = s.split(',')
            route_id = s[route_desc['route_id']]
            name = s[route_desc['route_short_name']].strip('\"')

            route_list[route_id] = Route(route_id, name)
            s = file.readline().strip('\n')

    trip_list = dict()
    trip_desc = dict()
    with TextIOWrapper(my_zip.open('trips.txt')) as file:
        s = file.readline().strip('\n').split(',')
        for i in range(len(s)):
            trip_desc[s[i]] = i

        s = file.readline().strip('\n')
        while s:
            s = s.split(',')
            trip_id = s[trip_desc['trip_id']]
            route_id = s[trip_desc['route_id']]
            service_id = s[trip_desc['service_id']]
            trip_headsign = s[trip_desc['trip_headsign']]
            direction_id = int(s[trip_desc['direction_id']])
            block_id = s[trip_desc['block_id']]

            trip_list[trip_id] = Trip(trip_id, route_id, service_id, trip_headsign, direction_id, block_id)
            s = file.readline().strip('\n')

    stop_times_desc = dict()
    with TextIOWrapper(my_zip.open('stop_times.txt')) as file:
        s = file.readline().strip('\n').split(',')
        for i in range(len(s)):
            stop_times_desc[s[i]] = i

        s = file.readline().strip('\n')
        while s:
            s = s.split(',')
            trip_id = s[stop_times_desc['trip_id']]
            arrival_time = s[stop_times_desc['arrival_time']]
            departure_time = s[stop_times_desc['departure_time']]
            stop_sequence = s[stop_times_desc['stop_sequence']]
            stop_id = s[stop_times_desc['stop_id']]

            if not arrival_time == departure_time:
                #print('arrival_time != departure_time : ' + str(s))
                raise Exception('arrival_time != departure_time : ' + str(s))

            trip_list[trip_id].add_stop(arrival_time, stop_sequence, stop_id)
            s = file.readline().strip('\n')
            
for trip_id in trip_list:
    trip_list[trip_id].finalise()

services = dict()
services['service_1'] = 'Workweek'
services['service_2'] = 'Saturday'
services['service_3'] = 'Sunday'

In [94]:
for route in route_list:
    print('line:',route_list[route].name)
    for rv in route_list[route].route_versions:
        print(route_list[route].route_versions[rv])
    print()

line: 1
11 x Brożka (nż) - Salwator
236 x Salwator - Wzgórza Krzesławickie
231 x Wzgórza Krzesławickie - Salwator
11 x Salwator - PT
22 x Jarzębiny - PH
17 x Elektromontaż (nż) - Wzgórza Krzesławickie

line: 19
12 x Łagiewniki - Borek Fałęcki
177 x Borek Fałęcki - Dworzec Towarowy
177 x Dworzec Towarowy - Borek Fałęcki
12 x Borek Fałęcki I - PT
6 x Elektromontaż (nż) - Dworzec Towarowy
6 x Politechnika - PH

line: 22
4 x Łagiewniki - Borek Fałęcki
157 x Borek Fałęcki - Wzgórza Krzesławickie
160 x Wzgórza Krzesławickie - Borek Fałęcki
7 x Borek Fałęcki I - PT
20 x Zajezdnia Nowa Huta - Kombinat
20 x Kombinat - Borek Fałęcki
20 x Borek Fałęcki - Kombinat
20 x Zajezdnia Nowa Huta - PH
3 x Brożka (nż) - Brożka (nż)
3 x Brożka (nż) - Wzgórza Krzesławickie

line: 3
16 x Łagiewniki - Nowy Bieżanów P+R
217 x Nowy Bieżanów P+R - Krowodrza Górka
217 x Krowodrza Górka - Nowy Bieżanów P+R
12 x Nowy Bieżanów P+R - Dworzec Towarowy
12 x Dworzec Towarowy - Nowy Bieżanów P+R
15 x Ćwiklińskiej - PT
6 x